# EDA and Cleaning The Data

## 1. Load the data

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
# Load data and explore
raw_data = pd.read_csv("../data/compas-scores-two-years.csv")
print(raw_data.shape[0])

7214


In [6]:
raw_data.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [3]:
print(raw_data['race'].unique())

['Other' 'African-American' 'Caucasian' 'Hispanic' 'Native American'
 'Asian']


## 2. Cleaning the data

In [5]:
# Filter data for only Caucasians and African-Americans
df = raw_data.loc[raw_data['race'].isin(['Caucasian', 'African-American']),
                  ['sex', 'age_cat', 'race', 'priors_count', 'c_charge_degree', 'c_jail_in', 'c_jail_out', 'two_year_recid']]

In [7]:
# Create new column for length of stay and remove NAs
df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])
df['length_of_stay'] = (df['c_jail_out'] - df['c_jail_in']).dt.days
df = df.dropna(subset=['length_of_stay'])

In [8]:
# Change variables into factors
df['sex'] = df['sex'].astype('category')
df['age_cat'] = df['age_cat'].map({"Less than 25": "< 25", "Greater than 45": "> 45"}).astype('category')
df['race'] = df['race'].astype('category')
df['c_charge_degree'] = df['c_charge_degree'].astype('category')
df['two_year_recid'] = df['two_year_recid'].astype('category')

In [9]:
# Perform normalization
df['priors_count'] = (df['priors_count'] - df['priors_count'].mean()) / df['priors_count'].std()
df['length_of_stay'] = (df['length_of_stay'] - df['length_of_stay'].mean()) / df['length_of_stay'].std()


In [10]:
print(df.shape[0])

5915


In [11]:
df.to_csv("../output/compas-scores-two-years_cleaned.csv", index=False)